In [ ]:
import sys
from collections import defaultdict
import re
from datetime import timedelta

ROS_DISTRO = 'rolling'
sys.path.insert(0, f'/opt/ros/{ROS_DISTRO}/lib/python3.121/site-packages')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor import Processor
from tracetools_analysis.processor.cpu_time import CpuTimeHandler
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.cpu_time import CpuTimeDataModelUtil
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

from multiprocessing import Pool

timer_periods = {"SensorA" : timedelta(milliseconds=25), "SensorB" : timedelta(milliseconds=40), "SensorC" : timedelta(milliseconds=50)}

def load_dropped_jobs(filename):
  with open(filename, 'r') as f:
    dropped_jobs = {}
    process_line = False
    for line in f:
      if line.startswith("Dropped jobs:"):
        process_line = True
        continue
      if process_line:
        name, stats = line.strip().split(":")
        dropped, total = [int(x) for x in stats.split("/")]
        dropped_jobs[name.strip()] = float(dropped) / float(total) if total > 0 else 0
    assert(process_line)
    return dropped_jobs
def load_inputs(string):
  return (load_file(directory_prefix + string), load_dropped_jobs(directory_prefix + string + '.log'))

def get_node_name(owner_info):
  node_name = owner_info.split(",")[0].split(":")[1].strip()
  return node_name

In [ ]:

# Load trace directory or converted trace file
directory_prefix = "data/"
events_dict = {}

def load_inputs(string):
  return (load_file(directory_prefix + string), load_dropped_jobs(directory_prefix + string + '.log'))

trace_names = ['trace-many-to-many.rm.60', 'trace-many-to-many.edf.60',
               'trace-many-to-many.events.60', 'trace-many-to-many.default.60',
               'trace-many-to-many.rm.80', 'trace-many-to-many.edf.80',
               'trace-many-to-many.events.80', 'trace-many-to-many.default.80',
               'trace-many-to-many.rm.90', 'trace-many-to-many.edf.90',
               'trace-many-to-many.events.90', 'trace-many-to-many.default.90']
display_names = ['RM,60%', 'EDF,60%', 'Events,60%', 'Default,60%',
                 'RM,80%', 'EDF,80%', 'Events,80%', 'Default,80%',
                 'RM,90%', 'EDF,90%', 'Events,90%', 'Default,90%']

# trace_names = ['sequences.rm.ro.uu', 'sequences.rm.re.uu',
#                'sequences.events.ro.uu', 'sequences.events.re.uu',
#                'sequences.default.uu',
#                'sequences.rm.ro.hu', 'sequences.rm.re.hu',
#                'sequences.events.ro.hu', 'sequences.events.re.hu',
#                'sequences.default.hu']
# display_names = ['RM (RO),60%', 'RM (RE),60%', 'Events (RO),60%', 'Events (RE),60%', 'Default,60%',
#                  'RM (RO),100%', 'RM (RE),100%', 'Events (RO),100%', 'Events (RE),100%', 'Default,100%']
# events_dict = {}
# for trace, display in zip(trace_names, display_names):
#   events_dict[display] = load_inputs(trace)

# events_dict["RM, (RO)"] =     (load_file(directory_prefix + 'timers-only-rm-ro'),     load_dropped_jobs(directory_prefix + 'timers-only-rm-ro.log'))
# events_dict["RM, (RE)"] =     (load_file(directory_prefix + 'timers-only-rm-re'),     load_dropped_jobs(directory_prefix + 'timers-only-rm-re.log'))
# events_dict["EDF, (RO)"] =    (load_file(directory_prefix + 'timers-only-edf-ro'),    load_dropped_jobs(directory_prefix + 'timers-only-edf-ro.log'))
# events_dict["EDF, (RE)"] =    (load_file(directory_prefix + 'timers-only-edf-re'),    load_dropped_jobs(directory_prefix + 'timers-only-edf-re.log'))
# events_dict["Events, (RO)"] = (load_file(directory_prefix + 'timers-only-events-ro'), load_dropped_jobs(directory_prefix + 'timers-only-events-ro.log'))
# events_dict["Events, (RE)"] = (load_file(directory_prefix + 'timers-only-events-re'), load_dropped_jobs(directory_prefix + 'timers-only-events-re.log'))
# events_dict["Default"] =      (load_file(directory_prefix + 'timers-only-default'),   load_dropped_jobs(directory_prefix + 'timers-only-default.log'))

dropped_df = pd.DataFrame({"Executor": [], "Utilization": [], "Node": [], "Drop Rate": []})

In [ ]:
callback_df = None
input_data = None
wcet_dict = {}

# TODO: Debug this, the dropped_jobs variable is empty
for trace, display in zip(trace_names, display_names):
  events = load_inputs(trace)
  utilization = display.split(",")[1]
  name = display.split(",")[0]
  dropped_jobs = events[1]
  if len(dropped_jobs) == 0:
    print("No dropped jobs found for " + name)
    continue
  for node, drop_rate in dropped_jobs.items():
    # temp_df = pd.DataFrame([[name, utilization, node, drop_rate]])
    dropped_df = pd.concat([pd.DataFrame([[name, utilization, node, drop_rate]], columns=dropped_df.columns), dropped_df], ignore_index=True)

  # # Timer manager processing
  # tid = 2750
  # trace_events = pd.DataFrame.from_dict(events[0])[['_name', 'timestamp', 'next_tid', 'prev_tid']]
  # sched_events = trace_events[(trace_events['next_tid']==tid) | (trace_events['prev_tid']==tid)]

  # Process
  if len(events[0]) == 0:
    print("No events found for " + name)
    continue
  handler = Ros2Handler.process(events[0])

  # Use data model utils to extract information
  data_util = Ros2DataModelUtil(handler.data)
  callback_symbols = data_util.get_callback_symbols()

  # if utilization != "90%":
  #   continue
  
  # callback_symbols = ros2_util.get_callback_symbols()
  for callback_object in callback_symbols.keys():
    owner_info = data_util.get_callback_owner_info(callback_object)
    if "parameter_events" in owner_info:
      continue
    owner_name = get_node_name(owner_info)
    temp_df = data_util.get_callback_durations(callback_object)
    temp_df["Executor"] = name
    temp_df["Node"] = owner_name
    temp_df["Utilization"] = utilization
    if callback_df is None:
      callback_df = temp_df
    else:
      callback_df = pd.concat([callback_df, temp_df], ignore_index=True)

    callback_durations = data_util.get_callback_durations(callback_object)[["duration"]].to_numpy(dtype=np.float64)[:-1] / 1000000.0
    
    if (owner_name + name) not in wcet_dict:
      wcet_dict[owner_name + name] = callback_durations.flatten()
    else:
      wcet_dict[owner_name + name] = np.concatenate((wcet_dict[owner_name + name], callback_durations.flatten())).flatten()

    # print(time_per_thread)
    # print(owner_info)
    # print(callback_durations)

total_drops_df = dropped_df[dropped_df["Node"] == "Total"]

In [ ]:
plt.figure(figsize=(7,4))
ax = sns.barplot(total_drops_df, y="Drop Rate", x="Executor", hue="Utilization", palette=["#003f5c", "#7393B3", "#7a7a7a"])
ax.set_title("Many to Many Topic, Uniprocessor")
ax.set_ylabel("Drop Rate")
ax.set_yscale('log')
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)

# ax.set_ylim([0, 1])
plt.savefig("dropped_jobs_many_to_many.svg", bbox_inches='tight')
plt.show()

wcet_grouped_dict = {}
for k in wcet_dict.keys():
  name = k.split("\n")[0]
  if name not in wcet_grouped_dict.keys():
    wcet_grouped_dict[name] = np.array([])
  # wcet_dict[k] = wcet_dict[k][wcet_dict[k] < 0.1]
  wcet_grouped_dict[name] = np.concatenate((wcet_grouped_dict[name], wcet_dict[k]))
sns.violinplot(wcet_grouped_dict)
plt.title("Many to Many, Uniprocessor")
plt.ylabel("WCET (ms)")
plt.show()

plt.figure(figsize=(30,4))
ax = sns.violinplot(wcet_dict)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
plt.title("Many to Many, Uniprocessor")
plt.ylabel("WCET (ms)")
plt.show()


# TODO: Pie chart of which types of jobs are dropped

In [ ]:
duration = 10
directory = "./data/autoware_benchmark/%ds/rmw_cyclonedds_cpp/" % duration
executors = ["autoware_default_edf", "autoware_default_rm", "autoware_default_events", "autoware_default_singlethreaded"]
dirs = [directory + e for e in executors]

files = [directory+'/std_output.log' for directory in dirs]

hot_path_name = None

# result maps each pair (exe, rmw) to lists of results corresponding to the runs
results = defaultdict(lambda: [])

hot_path_name_regex = re.compile(r'^ *hot path: *(.*)$')
hot_path_latency_regex = re.compile(r'^ *hot path latency: *(.+)ms \[min=(.+)ms, ' +
                                    r'max=(.+)ms, average=(.+)ms, deviation=(.+)ms\]$')
hot_path_drops_regex = re.compile(r'^ *hot path drops: *(.+) \[min=(.+), max=(.+), ' +
                                  r'average=(.+), deviation=(.+)\]$')
behavior_planner_period_regex = re.compile(r'^ *behavior planner period: *(.+)ms \[' +
                                            r'min=(.+)ms, max=(.+)ms, average=(.+)ms, ' +
                                            r'deviation=(.+)ms\]$')

rmw_regex = re.compile(r'^RMW Implementation: (rmw_.*)')
filename_regex = re.compile(r'.*/([0-9]+)s/(rmw_.*)/(.*)/std_output.log')
for count, file in enumerate(files):
    match = filename_regex.match(file)
    if not match:
        raise ValueError(f'File {file} does not conform to the naming scheme')

    extracted_duration, rmw, exe = match.groups()
    if int(extracted_duration) != duration:
        raise ValueError(f'File {file} does not match expected duration {duration}')
    with open(file) as fp:
        rmw_line, *data = fp.read().splitlines()

    match = rmw_regex.match(rmw_line)
    if match and rmw != match.groups()[0]:
        raise ValueError((f'{file}: mismatch between filename-rmw ("{rmw}")' +
                          f'and content-rmw("{match.groups()[0]}")'))

    if rmw not in file:
        raise ValueError(f'File {file} contains data from RMW {rmw}, contradicting its name')

    for line in data:
        match = hot_path_name_regex.match(line)
        if match:
            name, = match.groups()
            if hot_path_name is not None and hot_path_name != name:
                raise ValueError('Two different hotpaths in a single summary: ' +
                                  f'{name} {hot_path_name}')
            hot_path_name = name
            continue
        match = hot_path_latency_regex.match(line)
        if match:
            results[exe].append(float(match.groups()[0]))
            continue

if hot_path_name is None:
    raise RuntimeError('No hot_path defined in experiment.')

In [ ]:

# Set the style of the plot
# sns.set_style({'axes.facecolor':'white', 'grid.color': '.8'})
# sns.set_context("talk")  # Adjust this for larger or smaller text
# results["autoware_default_events"].sort()
# # results["autoware_default_fifo"].sort()
# results["autoware_default_rm"].sort()
# results["autoware_default_singlethreaded"].sort()
# results["autoware_default_staticsinglethreaded"].sort()
# print(results["autoware_default_events"][-5:])
# # print(results["autoware_default_fifo"][-5:])
# print(results["autoware_default_rm"][-5:])
# print(results["autoware_default_singlethreaded"][-5:])
# print(results["autoware_default_staticsinglethreaded"][-5:])

# Creating the violin plot with specific color scheme and settings
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
# ax = sns.boxplot(data=results, color="#2171b5", whis=100, linewidth=1.5, linecolor="#10385a",
#     fliersize=5, showfliers=False)
parts = plt.violinplot([results["autoware_default_singlethreaded"],
                     results["autoware_default_events"],
                     results["autoware_default_rm"],
                     results["autoware_default_edf"]],
                     positions=[0,1,2,3], showextrema=True)
# for pc in parts['bodies']:
#     pc.set_edgecolor('#ff0000')
ax = sns.violinplot(data=results, palette=["#2171b5", "#2171b5", "#2171b5", "#2171b5"],
                    linewidth=0, inner_kws={"box_width": 0, "whis_width": 0}, cut=0,
                    order=["autoware_default_singlethreaded", "autoware_default_events", "autoware_default_rm", "autoware_default_edf"])
# sns.boxplot(results, width=1, whis=100)

# Customizing the look and feel of the plot to match the bar graph
ax.set_ylabel("Latency (ms)", fontsize=16, labelpad=10)  # Y-axis Label
ax.set_title("Latency Summary {}s [FrontLidarDriver/RearLidarDriver -> ObjectCollision]".format(duration), fontsize=16, pad=20)  # Title

# Setting y-axis limits and labels similar to the bar chart
ax.set_ybound(0, 100)  # Y-axis Bounds
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))  # Major ticks every 10 units
ax.yaxis.set_minor_locator(ticker.MultipleLocator(2))   # Minor ticks every 2 units

# Enable grid only for major ticks on the y-axis
ax.grid(True, which='major', linestyle='-', linewidth=0.5)
ax.grid(True, which='minor', linestyle='', linewidth=0)

# Set axis labels
ax.set_yticklabels([int(x) for x in ax.get_yticks()], size=12)  # Y-axis Ticks
ax.set_xticklabels(["Default", "Events", "RM", "EDF"], ha="center", fontsize=16)

# Remove top and right borders for a cleaner look
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

# Show the plot
plt.savefig("latency_violin.svg")
plt.show()

In [ ]:
import os
import pandas as pd

root_dir = "data/"

samples = {}
df = pd.DataFrame(columns=["Executor", "Utilization", "Task", "Response Time"])

fifo_executors = ["Default", "Events"]
lifo_executors = ["RM", "EDF"]
utilizations = ["60%", "80%", "90%"]
task_df = pd.DataFrame(columns=["timestamp", "Executor", "Utilization", "Task", "release_time", "completion_time"])


for utilization in utilizations:
  for executor in fifo_executors:
    processor_x_queue = []
    processor_y_queue = []
    subset_df = callback_df[(callback_df["Executor"] == executor) & (callback_df["Utilization"] == utilization)]
    subset_df_sorted = subset_df.sort_values("timestamp")
    release_times = {"SensorA" : None, "SensorB" : None, "SensorC" : None}
    response_times_dir = "data/response_time.{}.{}/".format(executor.lower(), utilization.replace("%", ""))
    response_times_files = {"SensorA" : open(response_times_dir + "SensorA.node.txt", 'r'),
                            "SensorB" : open(response_times_dir + "SensorB.node.txt", 'r'),
                            "SensorC" : open(response_times_dir + "SensorC.node.txt", 'r'),
                            "ProcessorX": open(response_times_dir + "ProcessorX.node.txt", 'r'),
                            "ProcessorY": open(response_times_dir + "ProcessorY.node.txt", 'r')}
    for index, row in subset_df_sorted.iterrows():
      node_name = row["Node"].split("\n")[0]
      if "Sensor" in node_name:
        file_line = response_times_files[node_name].readline()
        sequence_number = int(file_line.split(":")[0])
        sensor_response_time = timedelta(microseconds=int(file_line.split(":")[1]) / 1000.0)
        completion_time = row["timestamp"] + row["duration"]
        release_time = completion_time - sensor_response_time
        
        processor_x_queue.append(row["timestamp"])
        processor_y_queue.append(row["timestamp"])
        
        # TODO: Get sequence number from file and save that in the task_df
        task_df = pd.concat([task_df, pd.DataFrame({"timestamp": [row["timestamp"]],
                                                    "Executor": [executor],
                                                    "Utilization": [utilization],
                                                    "Task": [node_name],
                                                    "release_time": [release_time],
                                                    "completion_time": [completion_time],
                                                    "children_executed": 0,
                                                    "sequence_number": sequence_number})])
        release_times[node_name] = release_time
      elif "Processor" in node_name:
        file_line = response_times_files[node_name].readline()
        parent_name = file_line.split(":")[0]
        sequence_number = int(file_line.split(":")[1])
        # TODO: Get the sequence number from the processor node file and retrieve the row corresponding to the provided sensor name and sequence number
        completion_time = row["timestamp"] + row["duration"]
        task_df.loc[(task_df['Task'] == parent_name) & (task_df['sequence_number'] == sequence_number) & (task_df["Utilization"] == utilization) & (task_df["Executor"] == executor), 'completion_time'] = completion_time
        task_df.loc[(task_df['Task'] == parent_name) & (task_df['sequence_number'] == sequence_number) & (task_df["Utilization"] == utilization) & (task_df["Executor"] == executor), 'children_executed'] += 1
      else:
        raise ValueError("Unknown node")
      
      # TODO: Iterate over events and measure time between each timer and its two children to find E2E latency of each task
      
  for executor in lifo_executors:
    processor_x_queue = []
    processor_y_queue = []
    subset_df = callback_df[(callback_df["Executor"] == executor) & (callback_df["Utilization"] == utilization)]
    subset_df_sorted = subset_df.sort_values("timestamp")
    release_times = {"SensorA" : None, "SensorB" : None, "SensorC" : None}
    response_times_dir = "data/response_time.{}.{}/".format(executor.lower(), utilization.replace("%", ""))
    response_times_files = {"SensorA" : open(response_times_dir + "SensorA.node.txt", 'r'),
                            "SensorB" : open(response_times_dir + "SensorB.node.txt", 'r'),
                            "SensorC" : open(response_times_dir + "SensorC.node.txt", 'r')}
    for index, row in subset_df_sorted.iterrows():
      node_name = row["Node"].split("\n")[0]
      if "Sensor" in node_name:
        file_line = response_times_files[node_name].readline()
        sequence_number = int(file_line.split(":")[0])
        sensor_response_time = timedelta(microseconds=int(file_line.split(":")[1]) / 1000.0)
        completion_time = row["timestamp"] + row["duration"]
        release_time = completion_time - sensor_response_time
        
        processor_x_queue.append(row["timestamp"])
        processor_y_queue.append(row["timestamp"])
        
        task_df = pd.concat([task_df, pd.DataFrame({"timestamp": [row["timestamp"]],
                                                    "Executor": [executor],
                                                    "Utilization": [utilization],
                                                    "Task": [node_name],
                                                    "release_time": [release_time],
                                                    "completion_time": [completion_time],
                                                    "children_executed": 0,
                                                    "sequence_number": sequence_number})])
        release_times[node_name] = release_time
      elif "Processor" in node_name:
        if "X" in node_name:
          sensor_timestamp = processor_x_queue.pop()
        elif "Y" in node_name:
          sensor_timestamp = processor_y_queue.pop()
        else:
          raise ValueError("Unknown processor")
        completion_time = row["timestamp"] + row["duration"]
        task_df.loc[task_df['timestamp'] == sensor_timestamp, 'completion_time'] = completion_time
        task_df.loc[task_df['timestamp'] == sensor_timestamp, 'children_executed'] += 1
      else:
        raise ValueError("Unknown node")
    


In [ ]:
# For each row in task_df, compute the difference of release_time and completion_time for each task and save that as response_time


task_df["Response Time"] = task_df.apply(lambda row: (row["completion_time"] - row["release_time"]) / timedelta(milliseconds=1), axis=1)

task_df["Response Time"] = task_df["Response Time"].astype(float)
cutoff = task_df.groupby(["Task", "Executor", "Utilization"])["Response Time"].quantile(0.997)
last_timestamps = task_df.groupby(["Task", "Executor", "Utilization"])["timestamp"].max()
cutoff_dict = cutoff.to_dict()

task_df["Cutoff"] = task_df.apply(lambda row: cutoff_dict[(row["Task"], row["Executor"], row["Utilization"])], axis=1)
task_df_filtered = task_df[(task_df["Response Time"] <= task_df["Cutoff"]) & (task_df["timestamp"] < task_df["completion_time"]) & (task_df["children_executed"] == 2)]


print("SensorA")
print(f"{'Executor':<15} {'60%':<10} {'80%':<10} {'90%':<10}")
print(f"{'-'*15} {'-'*10} {'-'*10} {'-'*10}")
print(f"{'Default':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'Events':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'EDF':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'RM':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print()
print("SensorB")
print(f"{'Executor':<15} {'60%':<10} {'80%':<10} {'90%':<10}")
print(f"{'-'*15} {'-'*10} {'-'*10} {'-'*10}")
print(f"{'Default':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorB') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'Events':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorB') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'EDF':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorB') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'RM':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorB') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print()
print("SensorC")
print(f"{'Executor':<15} {'60%':<10} {'80%':<10} {'90%':<10}")
print(f"{'-'*15} {'-'*10} {'-'*10} {'-'*10}")
print(f"{'Default':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorC') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Default') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'Events':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorC') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'Events') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'EDF':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorC') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'EDF') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")
print(f"{'RM':<15} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorC') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '60%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '80%')]['Response Time']):.3f} {'':<3} {max(task_df_filtered[(task_df_filtered['Task'] == 'SensorA') & (task_df_filtered['Executor'] == 'RM') & (task_df_filtered['Utilization'] == '90%')]['Response Time']):.3f}")



In [ ]:
plt.figure(figsize=(8,4))
ax = sns.violinplot(data=task_df_filtered[task_df_filtered["Task"] == "SensorA"], x="Executor", y="Response Time", hue="Utilization",
    palette=["#004d4c", "#008080", "#5ca3a3"], linewidth=0.5, cut=0, hue_order=["60%", "80%", "90%"],
    order=["Default", "Events", "EDF", "RM"],
    inner_kws={"box_width": 1, "whis_width": 0})
ax.set_title("Response Time, SensorA")
ax.set_ylabel("Response Time (ms)")
ax.set_ylim([0, 30])
ax.set_xticklabels(["Default", "Events", "EDF", "RM"], ha="center", rotation=30)

plt.savefig("response_times_sensora.svg", bbox_inches='tight')
plt.show()

plt.figure(figsize=(8,4))
ax = sns.violinplot(data=task_df_filtered[task_df_filtered["Task"] == "SensorB"], x="Executor", y="Response Time", hue="Utilization",
    palette=["#004d4c", "#008080", "#5ca3a3"], linewidth=0.5, cut=0, hue_order=["60%", "80%", "90%"],
    order=["Default", "Events", "EDF", "RM"],
    inner_kws={"box_width": 1, "whis_width": 0})
ax.set_title("Response Time, SensorB")
ax.set_ylabel("Response Time (ms)")
ax.set_ylim([0, 50])
ax.set_xticklabels(["Default", "Events", "EDF", "RM"], ha="center", rotation=30)

plt.savefig("response_times_sensorb.svg", bbox_inches='tight')
plt.show()

plt.figure(figsize=(8,4))
ax = sns.violinplot(data=task_df_filtered[task_df_filtered["Task"] == "SensorA"], x="Executor", y="Response Time", hue="Utilization",
    palette=["#004d4c", "#008080", "#5ca3a3"], linewidth=0.5, cut=0, hue_order=["60%", "80%", "90%"],
    order=["Default", "Events", "EDF", "RM"],
    inner_kws={"box_width": 1, "whis_width": 0})
ax.set_title("Response Time, SensorC")
ax.set_ylabel("Response Time (ms)")
ax.set_ylim([0, 60])
ax.set_xticklabels(["Default", "Events", "EDF", "RM"], ha="center", rotation=30)

plt.savefig("response_times_sensorc.svg", bbox_inches='tight')
plt.show()